<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Module5_Tutorial_GAMForClassifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Credit Card Default Case Study

Dani Bauer, 2022

In this tutorial, we will introduce one of the examples we will use in the coming tutorials: We will attempt to predict defaults on credits cards using a public dataset.

We will apply several of our predictive models from the past weeks (glm and gam) but also check the performance of the *naive Bayes classifier*. 

## Review of Concepts and Maths

As we saw in a previously, non-linear regression models are powerful approaches for depicting non-linear relationships -- with the key caveat that our explanatory variable had a single dimension only.  Once we venture into higher dimensions -- that means multiple, potentially interrelated features -- obtaining a similar fit will require a *very* large number of data points becaus of the **curse of dimensionality**.  Hence, statistical learning methods (need to) impose structure, in one way or another, and picking a learner in some way depends on picking an appropiate structure. **Generalized Additive Models** (GAMs) -- leverages the performance of non-linear regression models in lower dimensions but imposes an *additive* structure between the functions of the individual features:
$$
g\left(\mathbb{E}\left[Y | X_1,...,X_p\right]\right) = \alpha + f_1(X_1) + \ldots + f_p(X_p).
$$
As such, GAMs take on an intermediate position between linear regression and a general non-linear model:  They generalize the impact each feature can have on the outcome, but they keep the same structure on their relationship.  In particular, they do not allow for rich interactions between the variables -- which is the key downside of GAMs.

In the context of the binary classification problem discussed above, we can choose a logistic *link function* $g$ to predict the probability of success $p(X_1,...,X_p)$:
$$
\log\left\{\frac{p(X_1,...,X_p)}{1-p(X_1,...,X_p)}\right\} = \alpha + f(X_1,...,X_p). 
$$
but because of the **curse** this will require many many points.  A GAM assumes:
$$
\log\left\{\frac{p(X_1,...,X_p)}{1-p(X_1,...,X_p)}\right\} = \alpha + f_1(X_1) + \ldots + f_p(X_p). 
$$
Hence, the GAM takes on an intermediate position between logistic regression and general non-linear classification models.

As discussed, an alternative framing of binary classification is to model the feature probabilities conditional on class:
$$
f_j(X_1,...,X_p),\,j=1,2.
$$
but because of the **curse** learning the *joint* distribution will require many many points.  The Naive Bayes Classifier assumes:
$$
f_j(X_1,...,X_p) = f_{j1}(X_1)\times...\times f_{jp}(X_p),\,j=1,2,
$$
so that the distribuition of the features is conditionally independent.  This allows to focus on learning the *marginal* distribution, which -- much like for the GAM -- is much simpler because we can use all the data!  However, again this comes at a cost: the Naive Bayes classifier ignores interdependencies between features, which can be crucial in predictive modeling.

To summarize, GAMs and the Naive Bayes Classifiers finesse the **curse** by imposing structure on the interaction and the dependence structure, respectively.  However, of course that will limit their performance if these assumptions are highly inappropriate.


## Credit Card Default Application

We rely on the dataset `pa_data_UCI_Credit_Card.csv` from the UCI Machine Learning Repository (Lichman, M., 2013. [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science).  This datasets provides credit card defaults for customers in Taiwan.  We are given some demographic information ($X_1$-$X_5$), the previous history of payments ($X_6$-$X_{11}$), the amount of previous bills ($X_{12}$-$X_{17}$), and amounts of previous payments ($X_{18}$-$X_{23}$).  Finally, variable 24 is our target, whetyher there was a default in the next months.


As always, let's start with importing the libraries:

In [ ]:
!pip install pygam

In [25]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd 
from sklearn.model_selection import train_test_split
import seaborn as sns

from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_curve, auc
from sklearn import preprocessing
from sklearn.preprocessing import scale
from pygam import LogisticGAM, LinearGAM, GAM, s, f, l
from sklearn.naive_bayes import GaussianNB

Let's load the dataset

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [4]:
mydata = pd.read_csv('ML_656/UCI_Credit_Card.csv', index_col=0)

### Data Exploration and Preparation

In [ ]:
mydata.head()

Let's look at some aggregate statistics.

In [ ]:
mydata.describe()

First, a number of the variables are included numerically but really they have factor character, particularly Gender (1 = male; 2 = female), Education (1 = graduate school; 2 = university; 3 = high school; 4 = others), Marital status (1 = married; 2 = single; 3 = others), and default payment. Let's store them as factors.  We will do the same for history of past payment ($X_6$-$X_{11}$), although they really have ordinal character.

In [7]:
factor = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default.payment.next.month']

Also, a number of the levels occur very sparsely: there are 11 levels for all the `PAY` variables, but only the first six seem to be frequent.  So let's collapse levels 7 through 11 to one:

In [ ]:
mydata['PAY_0'][mydata['PAY_0']>4] = 4
mydata['PAY_2'][mydata['PAY_2']>4] = 4
mydata['PAY_3'][mydata['PAY_3']>4] = 4
mydata['PAY_4'][mydata['PAY_4']>4] = 4
mydata['PAY_5'][mydata['PAY_5']>4] = 4
mydata['PAY_6'][mydata['PAY_6']>4] = 4

Next, we rescale the numerical columns, as we know a number of learners require scaled inputs (and it should not matter for others):

In [9]:
mydata_numcols = mydata.drop(columns = factor)
mydata_faccols = mydata[factor].drop(columns = ['default.payment.next.month']).astype('category')
dummies = pd.get_dummies(mydata_faccols, drop_first=True)
mydata_numcols_sc_0 = scale(mydata_numcols)
mydata_numcols_sc = pd.DataFrame(data=mydata_numcols_sc_0, columns = mydata_numcols.columns, index = dummies.index)
mydata_sc = pd.concat([mydata_numcols_sc, dummies], axis = 1)
mydata_sc = pd.concat([mydata_sc, mydata['default.payment.next.month']], axis = 1)

And Let's relabel the long name of the dependent variable:

In [10]:
mydata = mydata.rename(columns={"default.payment.next.month": "default"})

Let's take a look:

In [ ]:
mydata.head()

In [ ]:
mydata.describe()

Let's check a correlation plot to make sure none of the variables is extremely correlated:

In [ ]:
mask = np.triu(np.ones_like(mydata.corr(), dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(mydata.corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

So it looks like the bill amounts are highly correlated.  Let's just keep the most recent one and then the average of all of them:


In [ ]:
mydata.insert(17, "BILL_AVG", (mydata['BILL_AMT1']+mydata['BILL_AMT2']+mydata['BILL_AMT3']+mydata['BILL_AMT4']+mydata['BILL_AMT5']+mydata['BILL_AMT6'])/6, True) 
mydata = mydata.rename(columns={"BILL_AMT1": "BILL_REC"})
del mydata['BILL_AMT2']
del mydata['BILL_AMT3']
del mydata['BILL_AMT4']
del mydata['BILL_AMT5']
del mydata['BILL_AMT6']
mydata.describe()

Let's save the dataset so that we can use it in coming tutorials without having to go through this procedure again:

In [15]:
mydata.to_csv('pa_data_UCI_Credit_Card_prepped.csv') 

### Predictive Modeling

As usually, let's split our dataset:

In [16]:
Train, Test = train_test_split(mydata, test_size=0.25)
Train_y = Train['default']
Train = Train.drop(columns = ['default'])
Test_y = Test['default']
Test = Test.drop(columns = ['default'])

Let's run a logistic regression model:

In [ ]:
logistic_model1 = LogisticRegression(fit_intercept=True, max_iter=500).fit(Train,Train_y)
print(logistic_model1.intercept_)
print(logistic_model1.coef_)

Let's check predictions:

In [ ]:
logistic_pred_1 = logistic_model1.predict_proba(Test)
np.sum(logistic_pred_1[:,1] > 0.5)
np.sum(logistic_pred_1[:,1] > 0.38)
logistic_pred_1_lab = logistic_pred_1[:,1] > 0.36
confusion_matrix(Test_y, logistic_pred_1_lab)

So we are missing quite a few.  Let's condider the AUC:

In [ ]:
fpr, tpr, threshold = roc_curve(Test_y, logistic_pred_1[:,1])
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Let's see if we can build a GAM that beats the linear learners.  We use some of the information from the GLM, and ignore some of the insignificant variables.  We include non-linear effects in the significant continuous features, and also interact `PAY_0` with `LIMIT_BAL` -- simply because that seems to be a good idea.  Remember, incorporating intuition is legitimate and even important given the *No Free Lunch Theorem* (of course, we could improve on the GLMs as well by using interaction terms).

In [ ]:
gam = LogisticGAM(s(0,n_splines=5) + l(1) + f(2) + f(3) + l(4) + f(5) + f(6) + f(7) + f(8) + f(9) + f(10) + s(11, n_splines=5) + s(12, n_splines=5) + s(13, n_splines=5) + l(14) + l(15) + l(16)).fit(Train, Train_y)
XX = gam.generate_X_grid
plt.rcParams['figure.figsize'] = (28, 8)
fig, axs = plt.subplots(1, 17)
titles = list(Train.columns)
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    ax.set_title(titles[i]);
plt.show()

Let's check if it does better:

In [ ]:
gam_preds = gam.predict_proba(Test)
gam_pred_labels = np.zeros(len(Test_y))
gam_pred_labels[gam_preds >0.5] = 1
confusion_matrix(Test_y, gam_pred_labels)

Looks like it!

Let's check the AUC:

In [ ]:
fpr, tpr, threshold = roc_curve(Test_y, gam_preds)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Finally, let's try a Naive Bayes Classifier:

In [ ]:
nb_model1 = GaussianNB()
nb_model1 = nb_model1.fit(Train,Train_y)
nb_pred_1 = nb_model1.predict(Test)
confusion_matrix(Test_y, nb_pred_1)

Doesn't look promising...

Let's check the AUC:

In [ ]:
nb_pred_1 = nb_model1.predict_proba(Test)
fpr, tpr, threshold = roc_curve(Test_y, nb_pred_1[:,1])
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

So it doesn't perform too well here.